# Реализация NLTK preprocessing for Transformer

In [48]:
import os
import pandas as pd
import nltk
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import re

In [34]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
from google.colab import files
uploaded = files.upload()

Saving transformer_df.csv to transformer_df.csv


In [21]:
df = pd.read_csv("transformer_df.csv")
df["Reviews"] = df["Reviews"].apply(lambda x: " ".join(eval(x)).lower())
df["Reviews"] = df["Reviews"].apply(lambda x: sent_tokenize(x))
df['Reviews'][0]

['кроссовки удобные, но быстро порвались.',
 'лёгкие, но скользкие на мокром асфальте.',
 'размер в размер, не натирают.',
 'через месяц начала отклеиваться подошва.',
 'отличные, ношу каждый день!']

In [30]:
def remove_stopwords(text):
    words = word_tokenize(text)  # Токенизация по словам
    filtered_words = [word for word in words if word not in stop_words or word in exceptions]
    return " ".join(filtered_words).replace(" ,", ",").replace(" .", ".").replace(" !", "!")

stop_words = set(stopwords.words('russian'))
exceptions = {
    "не", "ни", "нет", "да", "или", "и", "так", "потому", "что", "когда",
    "где", "кто", "что", "как", "зачем", "почему", "будто", "бы", "вдруг",
    "пусть", "тоже", "все", "всё", "этот", "та", "такой", "сам", "только",
    "другой", "ещё", "возможно", "почему", "впрочем", "хотя", "тем", "вместо",
    "между", "также", "тогда", "вдобавок", "после", "перед", "наоборот",
    "поэтому", "то", "тогда", "когда", "если", "даже", "включая", "совсем",
    "именно", "через", "сейчас", "когда", "будет", "для", "от", "среди"
}

df["Reviews"] = df["Reviews"].apply(lambda sentences: [remove_stopwords(sentence) for sentence in sentences])
df["Reviews"][0]

['кроссовки удобные, быстро порвались.',
 'лёгкие, скользкие мокром асфальте.',
 'размер размер, не натирают.',
 'через месяц начала отклеиваться подошва.',
 'отличные, ношу каждый день!']

In [40]:
!python -m spacy download ru_core_news_sm
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [42]:
nlp = spacy.load('ru_core_news_sm')

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc]
    return " ".join(lemmatized_words)

df["Reviews"] = df["Reviews"].apply(lambda sentences: [lemmatize_text(sentence) for sentence in sentences])
df["Reviews"][0]

['кроссовки удобный , быстро порваться .',
 'лёгкий , скользкий мокром асфальт .',
 'размер размер , не натирать .',
 'через месяц начать отклеиваться подошва .',
 'отличный , носить каждый день !']

In [44]:
df["Reviews"][0]

['кроссовки удобный , быстро порваться .',
 'лёгкий , скользкий мокром асфальт .',
 'размер размер , не натирать .',
 'через месяц начать отклеиваться подошва .',
 'отличный , носить каждый день !']

In [56]:
def remove_punctuation(text):
    exceptions_punctuation = {"!", "?", "-"}
    exclude_punctuation = "".join([p for p in string.punctuation if p not in exceptions_punctuation])
    text = text.translate(str.maketrans("", "", exclude_punctuation))
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["Reviews"] = df["Reviews"].apply(lambda sentences: [remove_punctuation(sentence) for sentence in sentences])
df["Reviews"][0]

['кроссовки удобный быстро порваться',
 'лёгкий скользкий мокром асфальт',
 'размер размер не натирать',
 'через месяц начать отклеиваться подошва',
 'отличный носить каждый день']

In [55]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

KeyError: "['Unnamed: 0'] not found in axis"

In [57]:
df.to_csv('preprocessed_transformer.csv', index=False)
files.download('preprocessed_transformer.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>